Read the Yolo detection results, denormalize bboxes

In [ ]:
def sort_key(s):
    # Extract numeric part from the string
    numeric_part = re.findall(r'\d+', s)
    
    # If numeric part is found, return it as integer, else print the filename and return 0
    if numeric_part:
        return int(numeric_part[0])
    else:
        print(f"No numeric part in filename: {s}")
        return 0

# Sort filenames before loading
image_dir = "Model_to_detect_3_classes_simplified_HSV_150epoch/test_images"
label_dir = "Model_to_detect_3_classes_simplified_HSV_150epoch/test_images_bm_labels"


# Ensure directories exist
if not os.path.exists(image_dir):
    print(f"Image directory does not exist: {image_dir}")
    exit(1)

if not os.path.exists(label_dir):
    print(f"Label directory does not exist: {label_dir}")
    exit(1)

# Get sorted lists of image and label filenames
image_filenames = sorted(os.listdir(image_dir), key=sort_key)
label_filenames = sorted(os.listdir(label_dir), key=sort_key)

# Verify that there is a 1-to-1 correspondence between images and labels
if len(image_filenames) != len(label_filenames):
    print("Number of image files does not match number of label files.")
    exit(1)

for image_file, label_file in zip(image_filenames, label_filenames):
    image_basename, _ = os.path.splitext(image_file)
    label_basename, _ = os.path.splitext(label_file)

    if image_basename != label_basename:
        print(f"Mismatch: image file {image_file} does not match label file {label_file}")
        exit(1)

# Load images and labels in the sorted order
mask_images_3channels = []
mask_images_1channel = []
RGB_images = []
GRAY_images = []
labels = []

for image_file, label_file in zip(image_filenames, label_filenames):
    # Load image
    imgpath = os.path.join(image_dir, image_file)
    img = cv2.imread(imgpath)
    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    RGB_images.append(rgb_image)
    mask1 = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype = np.uint8)
    mask2 = np.zeros((img.shape[0], img.shape[1]), dtype = np.uint8)
    mask_images_3channels.append(mask1)
    mask_images_1channel.append(mask2)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    GRAY_images.append(gray_image)

    # Load label
    txtpath = os.path.join(label_dir, label_file)
    with open(txtpath, 'r') as f:
        lines = f.readlines()
        labels.append(lines)

print("Loaded all images and labels in matching order.")
            
for i in range(len(mask_images_1channel)):
    for label in labels[i]:
        coords = [float(coords) for coords in re.findall(r'-?\d+\.?\d*', label)]
        label, x_center, y_center, bbox_width, bbox_height = coords
        x_min = np.int_(mask_images_1channel[i].shape[1] * (x_center - bbox_width/2))
        y_min = np.int_(mask_images_1channel[i].shape[0] * (y_center - bbox_height/2))
        x_max = np.int_(mask_images_1channel[i].shape[1] * (bbox_width + x_min/mask_images_1channel[i].shape[1]))
        y_max = np.int_(mask_images_1channel[i].shape[0] * (bbox_height + y_min/mask_images_1channel[i].shape[0]))
        mask_images_1channel[i] = cv2.rectangle(mask_images_1channel[i], pt1 = (x_min, y_min), pt2 = (x_max, y_max),color = (255,255,255), thickness = -1)
        mask_images_3channels[i] = cv2.rectangle(mask_images_3channels[i], pt1 = (x_min, y_min), pt2 = (x_max, y_max),color = (255,255,255), thickness = -1)

In [ ]:
print(f"Processed {len(RGB_images)} images")

In [ ]:
labels[4]

In [ ]:
# Generate a random index
index = random.choice(range(len(RGB_images)))
index = 3

# Add weighted images to create a single image with bounding boxes
dst = cv2.addWeighted(RGB_images[index], 0.5, mask_images_3channels[index], 0.5, 0)

# Plot the image
plt.figure(figsize=(10,10))
plt.imshow(dst)
plt.axis('off')
plt.show()

Create and Save eroded masks

In [ ]:
# Create a new directory for eroded masks
os.makedirs('Model_to_detect_3_classes_simplified_HSV_150epoch/test_eroded_masks', exist_ok=True)

# Initialize an empty list for eroded masks
eroded_masks = []

# Define the kernel for erosion operation
kernel = np.ones((11, 11), np.uint8)

# Loop over masks
for i in range(len(mask_images_1channel)):
    # Apply mask on image (this assumes mask and image are the same size)
    mask = mask_images_1channel[i] * (RGB_images[i][:,:,0] > 0)
    #mask = mask_images_1channel[i]
    # Erode the mask
    eroded_mask = cv2.erode(mask, kernel)
    
    # Add the eroded mask to the list
    eroded_masks.append(mask)

    # Save the eroded mask to file
    cv2.imwrite(f'Model_to_detect_3_classes_simplified_HSV_150epoch/test_eroded_masks/eroded_mask_{i}.png', eroded_mask)

In [ ]:
plt.imshow(eroded_masks[3])

In [ ]:
def make_background_white(image, lower_threshold, upper_threshold):
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, foreground_mask = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY)
    background_mask = cv2.bitwise_not(foreground_mask)
    image_with_white_background = image.copy()
    image_with_white_background[np.where(background_mask == 255)] = [255, 255, 255]

    white_background_image = Image.fromarray(image_with_white_background)

    return white_background_image

def make_background_black(image, edge_threshold=100):

    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    edges = cv2.Canny(gray_image, edge_threshold, edge_threshold * 3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray_image)

    # Fill the largest contour (assumed to be the background) with white
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [largest_contour], 0, 255, -1)

    image_with_black_background = image.copy()
    image_with_black_background[mask == 0] = [0, 0, 0]
    
    image_with_black_background = cv2.cvtColor(image_with_black_background, cv2.COLOR_RGB2BGR)
    black_background_image = Image.fromarray(image_with_black_background)

    return black_background_image

def keep_white_pixels(image, lower_threshold=(200, 200, 200), upper_threshold=(255, 255, 255)):
    
    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    mask = cv2.inRange(image, lower_threshold, upper_threshold)

    black_image = np.zeros_like(image)
    black_image[mask == 255] = image[mask == 255]
    gray_image = cv2.cvtColor(black_image, cv2.COLOR_RGB2GRAY)
    white_pixels_image = Image.fromarray(gray_image)

    return white_pixels_image

In [ ]:
# Create a new directory for first iteration inpainting
os.makedirs('Model_to_detect_3_classes_simplified_HSV_150epoch/test_images_W', exist_ok=True)

# Iterate over each image in RGB_images
for i, img in enumerate(RGB_images):
    # Apply the function
    white_background_image = make_background_white(img, 50, 255)

    # Save the image to file
    white_background_image.save(f'Model_to_detect_3_classes_simplified_HSV_150epoch/test_images_W/test_images_W_{i}.png')

Prepare images for 2nd inpainting iteration

In [ ]:
os.makedirs('Model_to_detect_3_classes_simplified_HSV_150epoch/test_second_iter_masks', exist_ok=True)

def sort_key(s):
    return int(re.findall(r'\d+', s)[0])

# Specify the directory of the images
directory = "Model_to_detect_3_classes_simplified_HSV_150epoch/test_first_iter_inpainting_results"

# Get list of all files in the specified directory and sort them
files = os.listdir(directory)
files = sorted(files, key=sort_key)

# Full paths to files
full_paths = [os.path.join(directory, file) for file in files]

# Iterate over each file in the directory
for i, file in enumerate(full_paths):
    # Apply the functions
    black_background_image = make_background_black(file)
    white_pixels_image = keep_white_pixels(np.array(black_background_image))

    # Save the image to file
    white_pixels_image.save(f'Model_to_detect_3_classes_simplified_HSV_150epoch/test_second_iter_masks/mask_image_{i}.png')

Prepare images for third inpainting (if needed)

In [ ]:
def make_background_black_updated(image, edge_threshold=100, frame_size=10):

    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    # Add frame around the image
    height, width, _ = image.shape
    framed_image = cv2.copyMakeBorder(image, frame_size, frame_size, frame_size, frame_size, cv2.BORDER_CONSTANT, value=[255, 255, 255])

    gray_image = cv2.cvtColor(framed_image, cv2.COLOR_RGB2GRAY)

    edges = cv2.Canny(gray_image, edge_threshold, edge_threshold * 3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray_image)

    # Fill the largest contour (assumed to be the background) with white
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [largest_contour], 0, 255, -1)

    framed_image_with_black_background = framed_image.copy()
    framed_image_with_black_background[mask == 0] = [0, 0, 0]

    # Remove the frame
    image_with_black_background = framed_image_with_black_background[frame_size:-frame_size, frame_size:-frame_size]
    
    image_with_black_background = cv2.cvtColor(image_with_black_background, cv2.COLOR_RGB2BGR)
    black_background_image = Image.fromarray(image_with_black_background)

    return black_background_image

In [ ]:
def make_background_black_updated_2(image, edge_threshold=100, frame_size=10):

    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    # Add frame around the image
    height, width, _ = image.shape
    framed_image = cv2.copyMakeBorder(image, frame_size, frame_size, frame_size, frame_size, cv2.BORDER_CONSTANT, value=[255, 255, 255])

    gray_image = cv2.cvtColor(framed_image, cv2.COLOR_RGB2GRAY)

    edges = cv2.Canny(gray_image, edge_threshold, edge_threshold * 3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray_image)

    # Fill the largest contour (assumed to be the background) with white
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [largest_contour], 0, 255, -1)

    framed_image_with_black_background = framed_image.copy()
    # Apply the mask to the color image
    for i in range(3):  # For each color channel
        framed_image_with_black_background[:, :, i][mask == 0] = 0

    # Remove the frame
    image_with_black_background = framed_image_with_black_background[frame_size:-frame_size, frame_size:-frame_size]

    return image_with_black_background


In [ ]:
def make_background_black_updated_3(image, edge_threshold=100, frame_size=10, min_contour_area=500):

    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    # Add frame around the image
    height, width, _ = image.shape
    framed_image = cv2.copyMakeBorder(image, frame_size, frame_size, frame_size, frame_size, cv2.BORDER_CONSTANT, value=[255, 255, 255])

    gray_image = cv2.cvtColor(framed_image, cv2.COLOR_RGB2GRAY)

    edges = cv2.Canny(gray_image, edge_threshold, edge_threshold * 3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(gray_image)

    # Fill the contours with white, but only if they are large enough
    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:
            cv2.drawContours(mask, [contour], 0, 255, -1)

    framed_image_with_black_background = framed_image.copy()
    # Apply the mask to the color image
    for i in range(3):  # For each color channel
        framed_image_with_black_background[:, :, i][mask == 0] = 0

    # Remove the frame
    image_with_black_background = framed_image_with_black_background[frame_size:-frame_size, frame_size:-frame_size]

    return image_with_black_background


In [ ]:
os.makedirs('Model_to_detect_3_classes_simplified_HSV_150epoch/test__final_inpainted_results', exist_ok=True)

def sort_key(s):
    return int(re.findall(r'\d+', s)[0])

# Specify the directory of the images
directory = "Model_to_detect_3_classes_simplified_HSV_150epoch/test_second_iter_inpainting_results"

# Get list of all files in the specified directory and sort them
files = os.listdir(directory)
files = sorted(files, key=sort_key)

# Full paths to files
full_paths = [os.path.join(directory, file) for file in files]

# Iterate over each file in the directory
for i, file in enumerate(full_paths):
    # Apply the functions
    black_background_image = make_background_black_updated_3(file)
    
    black_background_image = Image.fromarray(black_background_image)

    # Save the image to file
    black_background_image.save(f'Model_to_detect_3_classes_simplified_HSV_150epoch/test__final_inpainted_results/inpainted_image_{i}.png')

In [ ]:
# Read the images
predicted_mask = cv2.imread("Model_to_detect_3_classes_simplified_HSV_150epoch/predicted_masks_with_black_marks/predicted_mask_37.png", 0)
object_mask = cv2.imread("Model_to_detect_3_classes_simplified_HSV_150epoch/predicted_masks_inpainted_objects/val_image_3.png", 0)

# Convert the grayscale image to RGB
object_mask_rgb = cv2.cvtColor(object_mask, cv2.COLOR_GRAY2RGB)

# Create the "cleaned" image
cleaned_mask = np.ones_like(object_mask_rgb) *255

# Assign the colors to the cleaned_mask
cleaned_mask[np.where(predicted_mask == 0)] = [0, 0, 0]  # Background (black)
cleaned_mask[np.where(predicted_mask == 2)] = [255, 255, 255]  # Fragment (white)
cleaned_mask[np.where(object_mask == 128)] = [128, 128, 128]  # Motif (original colors)

# Display the image
plt.imshow(cleaned_mask)
plt.show()
plt.imsave('cleaned_mask_37.png', cleaned_mask)